<a href="https://colab.research.google.com/github/atstuyuki/mediapipe/blob/main/mediapipe_shoulder_toCSV_20230101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# セットアップ

さまざまなカメラアングルから肩の外転角度を算出　訓練用のCSVを作成する

In [ ]:
#google driveのマウント
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

#誤検出が多かったので、min_detection_confidence　0.9に変更

In [ ]:
# ライブラリーのインストール
!pip install numpy==1.19.3
!pip install mediapipe
# 初期設定 holistic とPoseを用意する　検出感度は0.5＞＞変更可能
import mediapipe as mp
import cv2
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
# Initialize MediaPipe Holistic.
holistic = mp_holistic.Holistic(
    static_image_mode=True, min_detection_confidence=0.9)
pose= mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.9)
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(color=[225,0,200],thickness=2, circle_radius=1) #color=[B,G,R]で指定,線の太さと〇の色も指定可能

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.9 MB 4.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
jax 0.3.25 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
cmdstanpy 1.0.8 requires numpy>=1.21, but you have numpy 1.19.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.0 MB 1.2 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.12.6 which is incompatible.
tensorflow 2.9.2 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.


In [ ]:
#手元の動画をgoogle colabにupload

from google.colab import files
import os
uploaded = files.upload()
#uploadedは辞書型なのでファイル名を取り出し、絶対パスを取得してfullpathという変数を当てる
for k in uploaded.keys():
  print('file name:',k)
fullpath=os.path.abspath(k)
print('fullpath:',fullpath)

Saving Movie_00_(Movie_09_(A30-90)).mp4 to Movie_00_(Movie_09_(A30-90)).mp4
file name: Movie_00_(Movie_09_(A30-90)).mp4
fullpath: /content/Movie_00_(Movie_09_(A30-90)).mp4


正解の肩外転角度を入力 スライダーバーで10度刻みに指定

In [ ]:
true_angle = 90 #@param {type:"slider", min:0, max:180, step:10}


カメラの設置角度指定

In [ ]:
cam_angle= 30  #@param {type:"slider", min:0, max:45, step:15}

右手か左手を指定(is_right Yes=1, NO=0)

In [ ]:
is_right_hand=1  #@param ["0", "1"] {allow-input: true}

保存ファイル名を指定

In [ ]:
movie_name= "kusu2"#@param {type:"string"}

In [ ]:
cap = cv2.VideoCapture(fullpath) #動画の読み込み
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #幅
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #高
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) #FPS
fourcc = cv2.VideoWriter_fourcc('m','p','4','v') #mp4出力
out = cv2.VideoWriter('out.mp4', fourcc, frame_rate, (width, height))
print('width:',width)
print('height:',height)
print('frame rate:',frame_rate)

#3次元座標の角度計算の定義
def calculate_3D_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #0がx座標　１がｙ座標　２がｚ座標
    #ベクトルを定義 長さと内積からコサインを計算
    vec_a=a-b
    vec_c=c-b
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_c=np.linalg.norm(vec_c)
    inner_product=np.inner(vec_a, vec_c)
    cos=inner_product/(len_vec_a*len_vec_c)
    
    #角度（ラジアン）の計算
    rad=np.arccos(cos)
    
    #ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

#2次元からの角度計算の定義
def calculate_2D_angle(a,b,c):
  a=np.array(a)#firtst
  b=np.array(b)#second
  c=np.array(c)#third
  #[x,y,z]座標から[x,y]のみスライスしてベクトルを作成
  vec_a=a[0:2]-b[0:2]
  vec_c=c[0:2]-b[0:2]
  len_vec_a=np.linalg.norm(vec_a)
  len_vec_c=np.linalg.norm(vec_c)
  inner_product=np.inner(vec_a, vec_c)
  cos=inner_product/(len_vec_a*len_vec_c)
    
#角度（ラジアン）の計算
  rad=np.arccos(cos)
#ラジアンから度数へ変換
  angle=np.rad2deg(rad)
        
  return angle


width: 774
height: 510
frame rate: 30


各種測定関数の定義

In [ ]:
#体軸からの肩の外転角度測定
#3次元座標の角度計算の定義
def calculate_3D_shoulder_angle(a,b,c,d,e):
    a = np.array(a) # 右肘
    b = np.array(b) # 右肩
    c = np.array(c) # 左肩
    #c1=[b[0], c[1],b[2]]#右肩のx, 左肩のy、右肩のzを取得
    d = np.array(d) # 右股
    e = np.array(e) # 左股
    #0がx座標　１がｙ座標　２がｚ座標
    #中点を定義
    global shoulder_mid
    shoulder_mid = (b+c)/2 #肩の中心座標
    global hip_mid
    hip_mid = (d+e)/2 #股関節の中心座標　world系では0になるはず
    #ベクトルを定義 長さと内積からコサインを計算
    vec_a=hip_mid-shoulder_mid #体軸のベクトル
    vec_b=a-b
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_b=np.linalg.norm(vec_b)
    inner_product=np.inner(vec_a, vec_b)
    cos=inner_product/(len_vec_a*len_vec_b)
    
    #角度（ラジアン）の計算
    rad=np.arccos(cos)
    
    #ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

#2次元からの角度計算の定義
def calculate_2D_shoulder_angle(a,b,c,d,e):
    a = np.array(a) # 右肘
    b = np.array(b) # 右肩
    c = np.array(c) # 左肩
    #c1=[b[0], c[1],b[2]]#右肩のx, 左肩のy、右肩のzを取得
    d = np.array(d) # 右股
    e = np.array(e) # 左股
    #0がx座標　１がｙ座標　２がｚ座標
    #中点を定義
    shoulder_mid = (b+c)/2 #肩の中心座標
    hip_mid = (d+e)/2 #股関節の中心座標　world系では0になるはず
  #[x,y,z]座標から[x,y]のみスライスしてベクトルを作成
    vec_a=hip_mid[0:2]-shoulder_mid[0:2]#体軸のベクトル
    vec_b=a[0:2]-b[0:2]
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_b=np.linalg.norm(vec_b)
    inner_product=np.inner(vec_a, vec_b)
    cos=inner_product/(len_vec_a*len_vec_b)
    
#角度（ラジアン）の計算
    rad=np.arccos(cos)
#ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle

#面積　sizeの計算 2Dの3点から外積を算出　
def calc_size(a, b, c):
    a=np.array(a[0:2])#先端の座標1(x,yのみ)
    b=np.array(b[0:2])#先端の座標2
    c=np.array(c[0:2])#基部の座標

    vec1=a-c
    vec2=b-c
    
    size=np.cross(vec1,vec2)*10#数字が小さいので10倍しておく
    size=np.abs(size)#絶対値をとる
    return size


#pointの距離計測の関数 a-b（2点間の座標）間の距離を基準となる距離(c-dの距離）で除したものとする
def point_distance(a,b,c,d):
    a=np.array(a)#遠位関節の座標
    b=np.array(b)#近位関節の座標
    c=np.array(c)#基準関節の座標1
    d=np.array(d)#基準関節の座標2
    
    dist1=np.linalg.norm(a-b)
    dist2=np.linalg.norm(c-d)
    distance=dist1/dist2
    return distance 

# サンプルビデオの読み込み＋静止画に分解

In [ ]:
# video2images
import os
import shutil
import cv2

# 既にimagesフォルダーがあれば削除
if os.path.isdir('images'):
    shutil.rmtree('images')

os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= fullpath,   
                   image_dir='./images/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 1#適宜調整
    length = 3500
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

In [ ]:
#pose_landmarkのリストを取得して配列に変換

import numpy as np
landmarklist=[]#空の配列を作成
for index in range(len(mp_holistic.PoseLandmark)):
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_x')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_y')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_z')
    landmarklist.append(mp_holistic.PoseLandmark(index).name+'_visibility')
#リストをarrayに変換
landmarklist=np.array(landmarklist).reshape(1,-1)
#landmarkshape の内容の確認
print('landmark list data type:',landmarklist.dtype)
print('landmark list:',landmarklist)
print('landmark list shape:',landmarklist.shape)


landmark list data type: <U27
landmark list: [['NOSE_x' 'NOSE_y' 'NOSE_z' 'NOSE_visibility' 'LEFT_EYE_INNER_x'
  'LEFT_EYE_INNER_y' 'LEFT_EYE_INNER_z' 'LEFT_EYE_INNER_visibility'
  'LEFT_EYE_x' 'LEFT_EYE_y' 'LEFT_EYE_z' 'LEFT_EYE_visibility'
  'LEFT_EYE_OUTER_x' 'LEFT_EYE_OUTER_y' 'LEFT_EYE_OUTER_z'
  'LEFT_EYE_OUTER_visibility' 'RIGHT_EYE_INNER_x' 'RIGHT_EYE_INNER_y'
  'RIGHT_EYE_INNER_z' 'RIGHT_EYE_INNER_visibility' 'RIGHT_EYE_x'
  'RIGHT_EYE_y' 'RIGHT_EYE_z' 'RIGHT_EYE_visibility' 'RIGHT_EYE_OUTER_x'
  'RIGHT_EYE_OUTER_y' 'RIGHT_EYE_OUTER_z' 'RIGHT_EYE_OUTER_visibility'
  'LEFT_EAR_x' 'LEFT_EAR_y' 'LEFT_EAR_z' 'LEFT_EAR_visibility'
  'RIGHT_EAR_x' 'RIGHT_EAR_y' 'RIGHT_EAR_z' 'RIGHT_EAR_visibility'
  'MOUTH_LEFT_x' 'MOUTH_LEFT_y' 'MOUTH_LEFT_z' 'MOUTH_LEFT_visibility'
  'MOUTH_RIGHT_x' 'MOUTH_RIGHT_y' 'MOUTH_RIGHT_z'
  'MOUTH_RIGHT_visibility' 'LEFT_SHOULDER_x' 'LEFT_SHOULDER_y'
  'LEFT_SHOULDER_z' 'LEFT_SHOULDER_visibility' 'RIGHT_SHOULDER_x'
  'RIGHT_SHOULDER_y' 'RIGHT_SHOULDER_z' 

# MediaPipeで静止画を処理

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
import time
#変数の初期化
rtshoulderAnglelist=[]
rtshoulderAbductionlist=[]
rtshoulder_3DAnglelist=[]
rtshoulder_3DAbductionlist=[]

ltshoulderAnglelist=[]
ltshoulderAbductionlist=[]
ltshoulder_3DAnglelist=[]
ltshoulder_3DAbductionlist=[]

true_angle_list=[]
cam_angle_list=[]
is_right_hand_list=[]
timelist=[]

#各関節の3次元座標もリスト化する

"""
nose_x_list=[]
nose_y_list=[]
nose_z_list=[]
rtelbow_x_list=[]
rtelbow_y_list=[]
rtelbow_z_list=[]
rtshoulder_x_list=[]
rtshoulder_y_list=[]
rtshoulder_z_list=[]

ltelbow_x_list=[]
ltelbow_y_list=[]
ltelbow_z_list=[]
ltshoulder_x_list=[]
ltshoulder_y_list=[]
ltshoulder_z_list=[]

rthip_x_list=[]
rthip_y_list=[]
rthip_z_list=[]
lthip_x_list=[]
lthip_y_list=[]
lthip_z_list=[]
"""
#上腕の長さを体幹の長さで割ったものを変数とする
rtarmdist_ratiolist=[]
ltarmdist_ratiolist=[]
#上腕軸と体幹軸のベクトルの外積
rtarm_sizelist=[]
ltarm_sizelist=[]
#肩幅と体幹軸のベクトルの外積
rttrunk_sizelist=[]
lttrunk_sizelist=[]

data_land = np.zeros((0,132))

# image file names to files in list format
files=[]
for name in sorted(glob.glob('./images/*.png')):
    files.append(name)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in files}
w = width             # 画像の横幅を取得
h = height            # 画像の縦幅を取得

for name, image in images.items():
  # Convert the BGR image to RGB and process it with MediaPipe Pose.
  if image is None:
      break
  results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  landmarks = results.pose_landmarks
  if landmarks is None:
      print('検出できなかったフレーム番号:',name)
      continue
   #各検出ポイントをarrayに変換
  rtshoulder=[results.pose_world_landmarks.landmark[12].x,results.pose_world_landmarks.landmark[12].y,results.pose_world_landmarks.landmark[12].z]
  rtelbow=[results.pose_world_landmarks.landmark[14].x,results.pose_world_landmarks.landmark[14].y,results.pose_world_landmarks.landmark[14].z]
  #rtwrist=[results.pose_world_landmarks.landmark[16].x,results.pose_world_landmarks.landmark[16].y,results.pose_world_landmarks.landmark[16].z]
  rthip=[results.pose_world_landmarks.landmark[24].x,results.pose_world_landmarks.landmark[24].y,results.pose_world_landmarks.landmark[24].z]
  ltshoulder=[results.pose_world_landmarks.landmark[11].x,results.pose_world_landmarks.landmark[11].y,results.pose_world_landmarks.landmark[11].z]
  ltelbow=[results.pose_world_landmarks.landmark[13].x,results.pose_world_landmarks.landmark[13].y,results.pose_world_landmarks.landmark[13].z]
  lthip=[results.pose_world_landmarks.landmark[23].x,results.pose_world_landmarks.landmark[23].y,results.pose_world_landmarks.landmark[23].z]
  nose=[results.pose_world_landmarks.landmark[0].x,results.pose_world_landmarks.landmark[0].y,results.pose_world_landmarks.landmark[0].z]
  #midshoulder=(np.array(rtshoulder)+np.array(ltshoulder))/2
  #midhip=(np.array(rthip)+np.array(lthip))/2
  #3D座標から角度を計算 2Dか3Dか選ぶ
  #rtelbowAngleは右肩-右肘-右手関節
  #rtelbowAngle=180-calculate_2D_angle(rtshoulder,rtelbow,rtwrist)
  #rtelbow_3DAngle=180-calculate_3D_angle(rtshoulder,rtelbow,rtwrist)
  #rtshoulderAngleは左肩-右肩-右肘
  rtshoulderAngle=calculate_2D_angle(ltshoulder,rtshoulder,rtelbow)
  rtshoulder_3DAngle=calculate_3D_angle(ltshoulder,rtshoulder,rtelbow)

  ltshoulderAngle=calculate_2D_angle(rtshoulder,ltshoulder,ltelbow)
  ltshoulder_3DAngle=calculate_3D_angle(rtshoulder,ltshoulder,ltelbow)  
  #rtshoulderAbductionは体軸ベクトル（両肩中心と両股中心を結ぶ線と右肩-右肘ベクトル）
  rtshoulderAbduction=calculate_2D_shoulder_angle(rtelbow,rtshoulder,ltshoulder, rthip, lthip)
  rtshoulder_3DAbduction=calculate_3D_shoulder_angle(rtelbow,rtshoulder,ltshoulder, rthip, lthip)

  ltshoulderAbduction=calculate_2D_shoulder_angle(ltelbow,ltshoulder,rtshoulder, rthip, lthip)
  ltshoulder_3DAbduction=calculate_3D_shoulder_angle(ltelbow,ltshoulder,rtshoulder, rthip, lthip)

  #距離の計算
  rtarmdist_ratio=point_distance(rtelbow, rtshoulder, rtshoulder,rthip)
  ltarmdist_ratio=point_distance(ltelbow,ltshoulder,ltshoulder, lthip)

  #面積の計算
  rtarm_size=calc_size(rtelbow, rthip,rtshoulder)
  ltarm_size=calc_size(ltelbow,lthip,ltshoulder)
#肩幅と体幹軸のベクトルの外積(先端、先端、基部の順)
  rttrunk_size=calc_size(ltshoulder,rthip,rtshoulder)
  lttrunk_size=calc_size(rtshoulder,lthip,ltshoulder)
#リストに計算した変数を追加してゆく
  rtshoulderAnglelist.append(rtshoulderAngle)
  rtshoulderAbductionlist.append(rtshoulderAbduction)
  rtshoulder_3DAnglelist.append(rtshoulder_3DAngle)
  rtshoulder_3DAbductionlist.append(rtshoulder_3DAbduction)

  ltshoulderAnglelist.append(rtshoulderAngle)
  ltshoulderAbductionlist.append(rtshoulderAbduction)
  ltshoulder_3DAnglelist.append(rtshoulder_3DAngle)
  ltshoulder_3DAbductionlist.append(rtshoulder_3DAbduction)

  true_angle_list.append(true_angle)
  cam_angle_list.append(cam_angle)
  is_right_hand_list.append(is_right_hand)
  timelist.append(time.time())
  #距離のリスト
  rtarmdist_ratiolist.append(rtarmdist_ratio)
  ltarmdist_ratiolist.append(ltarmdist_ratio)

  #面積の計算
  rtarm_sizelist.append(rtarm_size)
  ltarm_sizelist.append(ltarm_size)
#肩幅と体幹軸のベクトルの外積(先端、先端、基部の順)
  rttrunk_sizelist.append(rttrunk_size)
  lttrunk_sizelist.append(lttrunk_size)

  # Draw pose landmarks.
  annotated_image = image.copy()
  # Get Landmarks
  #landmarks=results.pose_landmarks.landmark
 
  """
  Holisticを使用しない場合はこの部分をコメントアウト
  
  mp_drawing.draw_landmarks(annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.face_landmarks, 
      connections=mp_holistic.FACEMESH_TESSELATION,
      landmark_drawing_spec=drawing_spec,
      connection_drawing_spec=drawing_spec)
  """
  mp_drawing.draw_landmarks(
        image=annotated_image, 
        landmark_list=results.pose_landmarks, 
        connections=mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=drawing_spec,
        connection_drawing_spec=drawing_spec)
  
  #get coordinate poseの33か所の3次元座標データと確信度(visibilisty)をすべて結合してCSVに変換
  data_land2 = np.zeros((1,3))
    
  for x in range (0,33):#pose_landmarksかpose_world_landmarksか選ぶ
      data1 = results.pose_world_landmarks.landmark[x].x
      data2 = results.pose_world_landmarks.landmark[x].y
      data3 = results.pose_world_landmarks.landmark[x].z
      data4 = results.pose_world_landmarks.landmark[x].visibility
      keydata = np.hstack((data1,data2,data3,data4)).reshape(1,-1)
      data_land2 = np.hstack((data_land2,keydata))

  data_land2 = data_land2[:,3:]
  data_land = np.vstack((data_land,data_land2))

  final_data= np.vstack((landmarklist,data_land))
  
  cv2.imwrite(name, annotated_image)  

#savetxtではデータの型が合わないと保存でき無いことがあるのでfmtで文字の型を指定
  np.savetxt('keypoint_results.csv',final_data,delimiter = ',',fmt = '%s')


In [ ]:
import pandas as pd
landmark_df = pd.read_csv('keypoint_results.csv')
display(landmark_df)

,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_z,LEFT_EYE_INNER_visibility,LEFT_EYE_x,LEFT_EYE_y,...,RIGHT_HEEL_z,RIGHT_HEEL_visibility,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_z,LEFT_FOOT_INDEX_visibility,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_visibility
0,0.027301,-0.645031,-0.209761,0.999941,0.024858,-0.682254,-0.188754,0.999895,0.025183,-0.682812,...,0.035736,0.826615,0.108240,0.842474,0.143947,0.962064,-0.015633,0.835769,-0.082171,0.976715
1,0.025432,-0.640995,-0.225777,0.999943,0.023830,-0.678197,-0.204952,0.999895,0.024154,-0.678749,...,0.040657,0.822575,0.109725,0.847150,0.135866,0.963424,-0.014458,0.839100,-0.078217,0.976057
2,0.024591,-0.644822,-0.219622,0.999951,0.023618,-0.681925,-0.198531,0.999909,0.023939,-0.682482,...,0.046740,0.841496,0.110546,0.841421,0.145729,0.968988,-0.015659,0.840605,-0.071079,0.979522
3,0.015924,-0.643897,-0.220052,0.999946,0.015342,-0.680829,-0.198800,0.999900,0.015651,-0.681375,...,0.029720,0.831985,0.112819,0.837532,0.151988,0.963947,-0.015334,0.833631,-0.091150,0.978456
4,0.021977,-0.648178,-0.217087,0.999947,0.020155,-0.685371,-0.196533,0.999898,0.020460,-0.685910,...,0.037209,0.851116,0.116853,0.837415,0.170651,0.970467,-0.008366,0.837870,-0.080799,0.982129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,0.044782,-0.627371,-0.213710,0.999961,0.042018,-0.665719,-0.194304,0.999930,0.042334,-0.666292,...,0.034219,0.855090,0.113919,0.844815,0.116494,0.975675,-0.009716,0.847806,-0.084568,0.982470
135,0.045273,-0.631883,-0.222183,0.999952,0.041196,-0.670255,-0.202541,0.999914,0.041532,-0.670816,...,0.037752,0.863909,0.113395,0.847412,0.122255,0.978968,-0.011446,0.846021,-0.082636,0.984972
136,0.041512,-0.639003,-0.219396,0.999950,0.037306,-0.677027,-0.198694,0.999913,0.037651,-0.677593,...,0.044198,0.861399,0.109667,0.844793,0.119915,0.977863,-0.017476,0.843601,-0.074565,0.984072
137,0.050742,-0.646831,-0.201126,0.999943,0.044779,-0.684465,-0.181071,0.999899,0.045115,-0.685032,...,0.022525,0.869272,0.116755,0.847282,0.111685,0.978032,-0.005865,0.844033,-0.095867,0.986307


In [ ]:
#zip関数でリストをまとめてデータフレーム化
import pandas as pd
zipped=zip(true_angle_list,cam_angle_list,is_right_hand_list,
rtshoulderAnglelist,rtshoulderAbductionlist,rtshoulder_3DAnglelist,
rtshoulder_3DAbductionlist,ltshoulderAnglelist,ltshoulderAbductionlist,
ltshoulder_3DAnglelist,ltshoulder_3DAbductionlist,
rtarmdist_ratiolist,ltarmdist_ratiolist,
rtarm_sizelist,ltarm_sizelist,rttrunk_sizelist,lttrunk_sizelist)
df = pd.DataFrame(zipped, columns = ['true_angle','cam_angle','is_right_hand',
'rtshoulderAngle','rtshoulderAbduction','rtshoulder_3DAngle',
'rtshoulder_3DAbduction','ltshoulderAngle','ltshoulderAbduction',
'ltshoulder_3DAngle','ltshoulder_3DAbduction',
'rtarmdist_ratio','ltarmdist_ratio',
'rtarm_size','ltarm_size','rttrunk_size','lttrunk_size'])

In [ ]:
#時間列を初期化してSeriesへ
#timelistの開始数字を0に変更
np.array(timelist[0])
#d=np.full(len(timelist),timelist[0])#すべての要素がtimelistの１番目の値でlen(timelist)の行列を作成
#timelistの時間を初期化（1番目の要素を0として行列の引き算
deltatime=np.array(timelist)-np.full(len(timelist),timelist[0])
deltatime = pd.Series(deltatime)
deltatime.name='deltatime'
#deltatimeとデータフレーム(df)の結合
df= pd.concat([deltatime, df],axis=1)
#最終データの概要を確認
display(df)

,deltatime,true_angle,cam_angle,is_right_hand,rtshoulderAngle,rtshoulderAbduction,rtshoulder_3DAngle,rtshoulder_3DAbduction,ltshoulderAngle,ltshoulderAbduction,ltshoulder_3DAngle,ltshoulder_3DAbduction,rtarmdist_ratio,ltarmdist_ratio,rtarm_size,ltarm_size,rttrunk_size,lttrunk_size
0,0.000000,90,30,1,150.529722,70.097958,151.485365,72.758108,150.529722,70.097958,151.485365,72.758108,0.440335,0.531909,1.037588,0.214171,1.451408,1.415082
1,0.106802,90,30,1,150.994162,69.901905,151.466237,72.413039,150.994162,69.901905,151.466237,72.413039,0.443915,0.534130,1.047170,0.209656,1.467611,1.436956
2,0.200725,90,30,1,151.768924,70.851753,152.449610,73.253215,151.768924,70.851753,152.449610,73.253215,0.440336,0.528827,1.057300,0.220375,1.487388,1.449082
3,0.311354,90,30,1,151.532614,70.946633,152.332774,73.266636,151.532614,70.946633,152.332774,73.266636,0.439478,0.536378,1.047877,0.217119,1.491793,1.455937
4,0.404823,90,30,1,151.882702,70.941493,152.659566,73.404094,151.882702,70.941493,152.659566,73.404094,0.434411,0.528865,1.049278,0.222344,1.491202,1.450129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,14.640155,90,30,1,148.727071,66.652727,150.013885,69.814864,148.727071,66.652727,150.013885,69.814864,0.436855,0.531852,0.987241,0.240341,1.453076,1.414896
135,14.764190,90,30,1,149.115983,67.478239,150.703850,71.490504,149.115983,67.478239,150.703850,71.490504,0.445160,0.537038,0.992893,0.220647,1.430916,1.391579
136,14.883646,90,30,1,149.464400,67.851163,151.423731,72.168517,149.464400,67.851163,151.423731,72.168517,0.450901,0.530367,1.005380,0.225627,1.434145,1.397951
137,15.002272,90,30,1,148.273631,65.024205,150.453824,69.267894,148.273631,65.024205,150.453824,69.267894,0.445255,0.537179,0.952647,0.242996,1.405908,1.375014


In [ ]:
#2つのデータフレームを結合
df_final=pd.concat([df, landmark_df], axis=1)
#最終データの概要を確認
display(df_final)

,deltatime,true_angle,cam_angle,is_right_hand,rtshoulderAngle,rtshoulderAbduction,rtshoulder_3DAngle,rtshoulder_3DAbduction,ltshoulderAngle,ltshoulderAbduction,...,RIGHT_HEEL_z,RIGHT_HEEL_visibility,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_z,LEFT_FOOT_INDEX_visibility,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_visibility
0,0.000000,90,30,1,150.529722,70.097958,151.485365,72.758108,150.529722,70.097958,...,0.035736,0.826615,0.108240,0.842474,0.143947,0.962064,-0.015633,0.835769,-0.082171,0.976715
1,0.106802,90,30,1,150.994162,69.901905,151.466237,72.413039,150.994162,69.901905,...,0.040657,0.822575,0.109725,0.847150,0.135866,0.963424,-0.014458,0.839100,-0.078217,0.976057
2,0.200725,90,30,1,151.768924,70.851753,152.449610,73.253215,151.768924,70.851753,...,0.046740,0.841496,0.110546,0.841421,0.145729,0.968988,-0.015659,0.840605,-0.071079,0.979522
3,0.311354,90,30,1,151.532614,70.946633,152.332774,73.266636,151.532614,70.946633,...,0.029720,0.831985,0.112819,0.837532,0.151988,0.963947,-0.015334,0.833631,-0.091150,0.978456
4,0.404823,90,30,1,151.882702,70.941493,152.659566,73.404094,151.882702,70.941493,...,0.037209,0.851116,0.116853,0.837415,0.170651,0.970467,-0.008366,0.837870,-0.080799,0.982129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,14.640155,90,30,1,148.727071,66.652727,150.013885,69.814864,148.727071,66.652727,...,0.034219,0.855090,0.113919,0.844815,0.116494,0.975675,-0.009716,0.847806,-0.084568,0.982470
135,14.764190,90,30,1,149.115983,67.478239,150.703850,71.490504,149.115983,67.478239,...,0.037752,0.863909,0.113395,0.847412,0.122255,0.978968,-0.011446,0.846021,-0.082636,0.984972
136,14.883646,90,30,1,149.464400,67.851163,151.423731,72.168517,149.464400,67.851163,...,0.044198,0.861399,0.109667,0.844793,0.119915,0.977863,-0.017476,0.843601,-0.074565,0.984072
137,15.002272,90,30,1,148.273631,65.024205,150.453824,69.267894,148.273631,65.024205,...,0.022525,0.869272,0.116755,0.847282,0.111685,0.978032,-0.005865,0.844033,-0.095867,0.986307


In [ ]:
#CSVに保存
df_final.to_csv(movie_name+'.csv',index=False)

# 処理した画像をmp4動画に変換


In [ ]:
# 既に output.mp4 があれば削除
import os
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

! ffmpeg -r 10 -i images/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
# --- mp4動画の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="85%" height="85%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

＊作成したmp4動画は、左側のファイル欄にある　./mediapipe/output.mp4 を右クリックするとダウンロードできます。